In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import requests
import json

%load_ext autoreload
%autoreload 2
%matplotlib inline

Importing the data

In [2]:
df = pd.read_csv('data/E-Scooter_Trips_-_2019_Pilot.csv', low_memory=False)

Taking a first look

In [3]:
df.head()

,Trip ID,Start Time,End Time,Trip Distance,Trip Duration,Accuracy,Start Census Tract,End Census Tract,Start Community Area Number,End Community Area Number,Start Community Area Name,End Community Area Name,Start Centroid Latitude,Start Centroid Longitude,Start Centroid Location,End Centroid Latitude,End Centroid Longitude,End Centroid Location
0,758e9d21-609f-5479-8e2c-5e8f64258202,07/01/2019 05:00:00 PM,07/01/2019 05:00:00 PM,421,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ff33490c-254a-5af2-9315-d6b2a45b07f7,06/29/2019 06:00:00 PM,06/29/2019 06:00:00 PM,6318,31,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,f8a25729-e853-40f3-9200-7eea9f9c45f2,09/16/2019 01:00:00 PM,09/16/2019 01:00:00 PM,77,732,10,NaN,NaN,NaN,25.0,NaN,AUSTIN,NaN,NaN,NaN,41.894101,-87.763112,POINT (-87.7631118242259 41.8941012961134)
3,11d42b99-e839-346c-11d4-2b99e839346c,06/24/2019 07:00:00 PM,06/24/2019 07:00:00 PM,917,359,10,NaN,NaN,25.0,25.0,AUSTIN,AUSTIN,41.894101,-87.763112,POINT (-87.7631118242259 41.8941012961134),41.894101,-87.763112,POINT (-87.7631118242259 41.8941012961134)
4,0c226130-0fa6-518f-8fe6-219b9fa5f42f,07/12/2019 07:00:00 PM,07/12/2019 07:00:00 PM,0,218,0,NaN,NaN,21.0,21.0,AVONDALE,AVONDALE,41.938666,-87.711211,POINT (-87.7112106378686 41.938665887698),41.938666,-87.711211,POINT (-87.7112106378686 41.938665887698)


Column information and non-null counts

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710839 entries, 0 to 710838
Data columns (total 18 columns):
Trip ID                        710839 non-null object
Start Time                     710839 non-null object
End Time                       710839 non-null object
Trip Distance                  710839 non-null int64
Trip Duration                  710839 non-null int64
Accuracy                       710839 non-null int64
Start Census Tract             382637 non-null float64
End Census Tract               382475 non-null float64
Start Community Area Number    613450 non-null float64
End Community Area Number      612923 non-null float64
Start Community Area Name      613450 non-null object
End Community Area Name        612923 non-null object
Start Centroid Latitude        613451 non-null float64
Start Centroid Longitude       613451 non-null float64
Start Centroid Location        613451 non-null object
End Centroid Latitude          612923 non-null float64
End Centroid Longitud

Converting 'Start Time' and 'End Time' to datetime objects

In [5]:
df['Start Time'] = df['Start Time'].astype('datetime64[ns]')
df['End Time'] = df['End Time'].astype('datetime64[ns]')

We'll be combining our scooter dataset with weather data from [meteostat](https://api.meteostat.net). First, we'll figure out the range of dates that we'll need from the api.

In [6]:
print(df['Start Time'].min(), df['End Time'].max(), sep=', ')

2019-06-15 04:00:00, 2019-10-30 15:00:00


Obtaining this data through the meteostat API is described in the "weather_data.ipynb" notebook.

---

### Joining Datasets

We'll now join the weather data that we obtained from meteostat with the scooter data.

In [ ]:
weather_df = pd.read_csv("")